In [20]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras

Load the necessary packages, especially the keras layers using hte layer names directly

In [39]:
# libraries
#import rpy2.robjects as robjects
import pandas as pd
import keras
import numpy as np
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam

In [40]:
# load the data
#path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
path = "/Users/maj/Dropbox/DSG17/DSG_2017/"
data   = pd.read_csv(path + "data/data_full.csv")

In [41]:
print(data.columns)

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'album_id',
       'context_type', 'media_duration', 'listen_type', 'user_gender',
       'user_age', 'is_listened', 'sample_id', 'dataset', 'time_lag',
       'session_id', 'song_session_position', 'first_flow', 'time_diff',
       'hours', 'genre_plays', 'genre_skips', 'artist_plays', 'artist_skips',
       'album_plays', 'album_skips', 'song_plays', 'song_skips',
       'user_ratio_flow', 'user_ratio_full', 'genre_ratio', 'artist_ratio',
       'song_ratio', 'platform_name1', 'platform_name2', 'platform_family1',
       'platform_family2'],
      dtype='object')


Separate the combined data into the training, test and unknown set

In [42]:
tr = data[data.dataset == 'train']
ts = data[data.dataset == 'test']
unknown = data[data.dataset == 'unknown']

In [43]:
# keep only Flow songs in the data
# this proves to predict better, but some information is lost
tr = tr.query("listen_type == 1")
print(tr.shape)
print(ts.shape)
print(tr.columns)

(2275140, 36)
(193239, 36)
Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'album_id',
       'context_type', 'media_duration', 'listen_type', 'user_gender',
       'user_age', 'is_listened', 'sample_id', 'dataset', 'time_lag',
       'session_id', 'song_session_position', 'first_flow', 'time_diff',
       'hours', 'genre_plays', 'genre_skips', 'artist_plays', 'artist_skips',
       'album_plays', 'album_skips', 'song_plays', 'song_skips',
       'user_ratio_flow', 'user_ratio_full', 'genre_ratio', 'artist_ratio',
       'song_ratio', 'platform_name1', 'platform_name2', 'platform_family1',
       'platform_family2'],
      dtype='object')


In [44]:
print(data.user_id.nunique())
print(tr.user_id.nunique())
print(data.artist_id.nunique())
print(tr.artist_id.nunique())
print(data.genre_id.nunique())
print(tr.genre_id.nunique())
print(data.media_id.nunique())
print(tr.media_id.nunique())
print(data.context_type.nunique())
print(tr.context_type.nunique())

13451
13451
16826
16826
1334
1334
55975
55975
34
34


The maximum value should be equal to the number of unique values - 1 (Python indexing)

In [45]:
max(tr.media_id)

55974

From the full feature set, create a subset of the data with the features that can be passed to the model directly. That is everything excluding the IDs and large factor variables.

In [46]:
# Create the data input matrix that can be passed to the keras model
# i.e. only numeric and without IDs and target variable
dropVars = ['dataset','user_id', 'artist_id', 'media_id', "genre_id", "album_id", "session_id", "is_listened", "sample_id"]
tr_data = tr[[column for column in tr.columns if column not in dropVars]].as_matrix()
ts_data = ts[[column for column in tr.columns if column not in dropVars]].as_matrix()
unknown_data = unknown[[column for column in tr.columns if column not in dropVars]].as_matrix()

The structure of the keras model is as follows:
- Input layers ot specify the size of the data that goes in the model
- (Embedding layer, i.e. lookup table layer that assigns 50 values to each level. These values are then trained to somehow capture the essence of this level.)
- Dense layer, i.e. fully connected neural net layers
- Output layer, i.e. Dense layer with only one result and sigmoid activation (for a result between 0 and 1)

In [47]:
# create an input layer with one row of IDs
user_in   = Input(shape = (1,), dtype='int64', name = "user_in")
song_in   = Input(shape = (1,), dtype='int64', name = "song_in")
artist_in = Input(shape = (1,), dtype='int64', name = "artist_in")
genre_in  = Input(shape = (1,), dtype='int64', name = "genre_in")
context_in = Input(shape = (1,), dtype='int64',   name = "context_in")

# Create an embedding assigning k latent factors to each ID
# These will be optimized
# A regulariztaion is added to avoid very large weights
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_genres  = tr.genre_id.nunique()
n_context = tr.context_type.nunique()

# The layer needs the number of the input levels and the number of values for each level
user_embedding = Embedding(n_users,   50, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
song_embedding = Embedding(n_songs,   50, input_length=1, embeddings_regularizer=l2(1e-5))(song_in)
artist_embedding = Embedding(n_artists, 20, input_length=1, embeddings_regularizer=l2(1e-5))(artist_in)
genre_embedding = Embedding(n_genres,  20, input_length=1, embeddings_regularizer=l2(1e-5))(genre_in)
context_embedding = Embedding(n_context, 50, input_length=1, embeddings_regularizer=l2(1e-5))(context_in)


In [48]:
# Data input doesn't need any embedding and can directly be passed to a Dense layer
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

In [49]:

# Specify what to do with the layers
# We want to multiply them into a 'rating' matrix
#
embedding_input = concatenate([user_embedding,  song_embedding, artist_embedding, genre_embedding])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(128, activation = "relu")(embedding_input)
embedding_dense = BatchNormalization()(embedding_dense)
#embedding_dense = Flatten()(embedding_dense)

#data_input = Flatten()(data_in)
data_dense = Dense(128, activation = "relu")(data_in)
data_dense = BatchNormalization()(data_dense)
# Make into a vector, i.e. drop 2D structure
# The 2D structure is important for e.g. CNN filters,
# but not necessary in a dense layer, I think
x = concatenate([embedding_dense, data_dense])
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
# Correct the standard devitation calculated from a batch
# to better fit the 'true' sd
x = BatchNormalization()(x)
# "Drop" each node at a training stage with a certain probability 
# then reinsert it after the training run
# Avoids overfitting and increases speed
x = Dropout(0.5)(x)
#x = Dropout(0.5)(Dense(128, activation='relu')(x))
#x = BatchNormalization()(x)
#x = Dense(64, activation='relu')(x) 
#x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)
#x = merge([x, ub], mode = 'sum')
#x = merge([x, sb], mode = 'sum') # Can this be included in the line above?

# Then we specify the model that we want to use
#
model = Model([user_in, song_in, artist_in, genre_in, data_in], output) # 
model.compile(Adam(0.001), loss="binary_crossentropy", metrics = ['accuracy'])

In [ ]:
# run the estimations
#
#
model.fit([tr.user_id, tr.media_id, tr.artist_id, tr.genre_id, tr_data], tr.is_listened,  #
validation_data = ([ts.user_id, ts.media_id, ts.artist_id, ts.genre_id, ts_data], ts.is_listened),
batch_size = 22814, epochs = 1)

Train on 2275140 samples, validate on 193239 samples
Epoch 1/1
 547536/2275140 [======>.......................] - ETA: 143s - loss: 0.8496 - acc: 0.5551

In [ ]:
# predict on unlabelled set
pred = pd.DataFrame()
pred["sample_id"] = unknown.sample_id
pred["is_listened"] = model.predict([unknown.userIdx, unknown.songIdx, unknown.artistIdx, unknown.genreIdx])
pred.head(5)

In [ ]:
# adding naive submission
naive = pd.read_csv(path + "submissions/naive_ratio_user.csv")
pred_mean = pred
pred_mean["is_listened"] = (pred["is_listened"] + naive["is_listened"])/2
pred_mean.head(5)

In [ ]:
# saving submissions
pred.to_csv(path + "submissions/deep_128_64_flow.csv", index = False)
pred.to_csv(path + "submissions/deep_128_64_flow_plus_ratio_user.csv", index = False)